In [3]:
import networkx as nx
import networkx as nx
import plotly.graph_objs as go
from preprocessing import Preprocessing as pp
from classes import transportnetwork as tn
import plotly.io as pio
from plotly.subplots import make_subplots
from networkviz.visualisation import *

G = pp.create_network_from_trailway("../../../data/Railway Data_JL.xlsx")
TN = tn.TransportNetwork(G)
connected_components = list(nx.connected_components(TN.graph))

Network creation: 


100%|██████████| 69638/69638 [00:03<00:00, 22646.10it/s]


#  Finalized metrics list:



## Number of nodes:
### The number of nodes in our network.
#### count_nodes:

In [4]:
def count_nodes(TN):
    # Compute the number of nodes in the network
    num_nodes = len(TN.graph.nodes())

    return num_nodes

# Example usage:
num_nodes = count_nodes(TN)
print("Number of nodes in the network:", num_nodes)


Number of nodes in the network: 2719


## Number of edges:
### The number of edges in our network.
#### count_edges:

In [5]:
def count_edges(TN):
    # Compute the number of edges in the network
    num_edges = len(TN.graph.edges())

    return num_edges

# Example usage:
num_edges = count_edges(TN)
print("Number of edges in the network:", num_edges)


Number of edges in the network: 6169


## Network Diameter:
### The network diameter is the longest shortest path between any two nodes in a connected graph, and it can be computed the network diameter using the nx.diameter() function.. Note that the graph must be connected for the diameter to be computed; otherwise, the function will raise a NetworkXError.

#### compute_network_diameter:

In [6]:
def compute_network_diameter(TN):
    # Check if the graph is connected
    if nx.is_connected(TN.graph):
        # Compute the network diameter
        diameter = nx.diameter(TN.graph)
    else:
        # If the graph is not connected, raise an error
        raise ValueError("The graph is not connected. Cannot compute the network diameter.")

    return diameter

# Example usage:
try:
    diameter = compute_network_diameter(TN)
    print("Network diameter:", diameter)
except ValueError as e:
    print(e)


Network diameter: 47


### We can aslo highlight the path of this diameter using the following function:
#### find_diameter_path:

In [15]:
def find_diameter_path(TN):
    if nx.is_connected(TN.graph):
        periphery_nodes = nx.periphery(TN.graph)
        diameter_path = None
        for node1 in periphery_nodes:
            for node2 in periphery_nodes:
                if node1 != node2:
                    shortest_path = nx.shortest_path(TN.graph, node1, node2)
                    if diameter_path is None or len(shortest_path) > len(diameter_path):
                        diameter_path = shortest_path
        return diameter_path
    else:
        raise ValueError("The graph is not connected. Cannot compute the diameter path.")

# Example usage:
try:
    diameter_path = find_diameter_path(TN)
    print("Diameter path:", " -> ".join(map(str, diameter_path)))
except ValueError as e:
    print(e)


Diameter path: 1600 -> 2296 -> 2309 -> 1660 -> 2210 -> 1222 -> 1314 -> 555 -> 241 -> 567 -> 282 -> 1700 -> 505 -> 2307 -> 375 -> 643 -> 2076 -> 2469 -> 548 -> 389 -> 390 -> 2058 -> 314 -> 1370 -> 574 -> 1298 -> 546 -> 164 -> 513 -> 2376 -> 537 -> 1645 -> 969 -> 970 -> 1124 -> 2297 -> 905 -> 2300 -> 566 -> 1716 -> 2298 -> 1967 -> 1989 -> 1502 -> 497 -> 1751 -> 613 -> 536


### And it is also possible to visualize the diameter of the network on a map using Plotly, using the following function.
#### plot_diameter_path_geo:

In [26]:
def plot_diameter_path_geo(TN, diameter_path):
    node_lat = []
    node_lon = []

    for node in TN.graph.nodes():
        lat, lon = TN.graph.nodes[node]['lat'], TN.graph.nodes[node]['lon']
        node_lat.append(lat)
        node_lon.append(lon)

    # Create the base network graph
    node_trace = go.Scattergeo(
        lat=node_lat,
        lon=node_lon,
        mode='markers',
        marker=dict(size=6, color='blue'),
        text=list(TN.graph.nodes()),
        hoverinfo='text'
    )

    # Highlight the diameter path nodes
    path_node_trace = go.Scattergeo(
        lat=[TN.graph.nodes[node]['lat'] for node in diameter_path],
        lon=[TN.graph.nodes[node]['lon'] for node in diameter_path],
        mode='markers',
        marker=dict(size=8, color='red'),
        text=diameter_path,
        hoverinfo='text'
    )

    # Create the layout and plot
    layout = go.Layout(
        geo=dict(
            scope='asia',
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
            resolution=50,
            lonaxis=dict(range=[70, 140]),
            lataxis=dict(range=[15, 55])
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(l=0, r=0, t=0, b=0)  # Set margins to 0
    )

    fig = go.Figure(data=[node_trace, path_node_trace], layout=layout)
    fig.show()

# Example usage:
try:
    diameter_path = find_diameter_path(TN)
    plot_diameter_path_geo(TN, diameter_path)
except ValueError as e:
    print(e)
